Reference: https://www.tensorflow.org/tutorials/text/text_classification_rnn  
チュートリアル：RNN を使ったテキスト分類をやってみる  
実装の復習、レイヤーの挙動を確認する

In [1]:
import tensorflow_datasets as tfds
import tensorflow as tf

In [2]:
# import matplotlib.pyplot as plt

# def plot_graphs(history, metric):
#     plt.plot(history.history[metric])
#     plt.plot(history.history['val_'+metric], '')
#     plt.xlabel("Epochs")
#     plt.ylabel(metric)
#     plt.legend([metric, 'val_'+metric])
#     plt.show()

In [3]:
dataset, info = tfds.load('imdb_reviews/subwords8k', with_info=True, as_supervised=True)
train_examples, test_examples = dataset['train'], dataset['test']

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteJZBVEP/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteJZBVEP/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteJZBVEP/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0. Subsequent calls will reuse this data.


In [4]:
encoder = info.features['text'].encoder

In [5]:
print('Vocabulary size: {}'.format(encoder.vocab_size))

Vocabulary size: 8185


In [6]:
sample_string = 'Hello TensorFlow.'

encoded_string = encoder.encode(sample_string)
print('Encoded string is {}'.format(encoded_string))

original_string = encoder.decode(encoded_string)
print('The original string: "{}"'.format(original_string))

Encoded string is [4025, 222, 6307, 2327, 4043, 2120, 7975]
The original string: "Hello TensorFlow."


In [46]:
BUFFER_SIZE = 10000
BATCH_SIZE = 1

In [47]:
# train_dataset = (train_examples.shuffle(BUFFER_SIZE).padded_batch(BATCH_SIZE, padded_shapes=([None], [])))
# test_dataset  = (test_examples.padded_batch(BATCH_SIZE, padded_shapes=([None], [])))
train_dataset = (train_examples.shuffle(BUFFER_SIZE).padded_batch(BATCH_SIZE))
test_dataset = (test_examples.padded_batch(BATCH_SIZE))

In [49]:
#train_dataset.take()
train_dataset

<DatasetV1Adapter shapes: ((None, None), (None,)), types: (tf.int64, tf.int64)>

In [50]:
print(next(iter(train_dataset.take(1))))

(<tf.Tensor: shape=(1, 188), dtype=int64, numpy=
array([[ 394, 5996,    8,   57,  126,    1,  107, 2586, 6767, 4690, 1510,
        6196, 1045,    2,  706, 4920, 8045,    2, 1593,    2, 1714,  150,
          32,   12,  129, 8002, 7968,  123,  160,  550,   28,  153,  594,
        7374,   21, 6141,   40, 7374,   21, 5559,  360,  327,   11, 1011,
        2803,   58,   19, 2165, 5804,    9, 1174, 1139, 1735,  201,    1,
        3733,    9, 1723, 1886, 5170,    2,    1,  170,   29, 1714,    5,
        2544, 3219, 1255,    6,  381,    5,   30,  285, 7830,  277, 6510,
           1, 2596,  140, 1670,   50, 5706, 7641,   64,  362, 7974, 3290,
          24,   10,   16,   10,   17,  173,    9,   84, 7393,  516, 4128,
           2,  413,  708, 8034,    2,   50, 2387, 7908,  540, 5361,  100,
        1129, 3670, 5427, 2686, 3737, 1049,    9, 1006,   20,    1, 2182,
         910, 5243, 2966,  309,    9, 4145, 6820,  269,    3,  133, 1481,
         935,   22,    4, 1130,  952, 8050,    2, 4906, 7974, 5

In [51]:
X, t = next(iter(train_dataset.take(1)))
print(X)
print(t)

tf.Tensor(
[[  19  118 1495 7190 7961   27   18 5422  282   22 3015  168 4127    2
  5270 7961 2864    2    5  328  225   13 1287   22    1 2934 7961    6
     1  250    3 1374 5364 8044  522 3774   40   43 3015  451  589    6
     1  606  522 3339    4 6092 7961 5016   13 1287   37   11 2816    1
   563  166    3 7084 7961 1495 7190 7961 1333 2611  104   28    5  116
  3435    2 3621  773 1989 1954  158   19  760  307  260 7968   21    1
   451  589   50    1  869 2220   20   63  264 2258 8030  137   19 5931
    31  949  216 7974 3773  170   48    1  118   32    5 1986  863   36
   161   61   52 3078  942 4957 7961 4501 8044 7975 3239   19 6541   13
    18  118 3353   36  256 1942    5  136 7974  521  170   11    1  118
    32    9  761   72   37 1552  187   29   33    1  226  170   48    1
   118 1495 7190 7961   65   24   10   16   10   17  519 2203    2  844
  2889  199   18 3773   20    4 4790 1540    2 4961 2014  168   22 3833
  7961  318 4600 3997  154  406  655   11    1   65  

In [57]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(encoder.vocab_size, 64),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, None, 64)          523840    
_________________________________________________________________
bidirectional_2 (Bidirection (None, 128)               66048     
_________________________________________________________________
dense (Dense)                (None, 64)                8256      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 598,209
Trainable params: 598,209
Non-trainable params: 0
_________________________________________________________________


In [53]:
embedding = tf.keras.layers.Embedding(encoder.vocab_size, 64)

In [54]:
out = embedding(X)
out

<tf.Tensor: shape=(1, 704, 64), dtype=float32, numpy=
array([[[-0.00243879, -0.03723186, -0.04148873, ..., -0.01110616,
         -0.04605542,  0.02107353],
        [-0.04139216,  0.04532498,  0.04176164, ..., -0.03528819,
         -0.00649023, -0.02422931],
        [ 0.00736023, -0.01917006,  0.0453595 , ...,  0.01845105,
         -0.0372736 ,  0.02823979],
        ...,
        [ 0.00920029,  0.01557652,  0.04441004, ...,  0.00344764,
         -0.01300721, -0.04172079],
        [ 0.04563669, -0.0262435 ,  0.02199098, ...,  0.02491287,
         -0.0229135 ,  0.0396852 ],
        [-0.00263027, -0.00501919, -0.0368566 , ...,  0.02392096,
         -0.00444164, -0.01272782]]], dtype=float32)>

In [55]:
bidirectional = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64))
bidirectional

In [56]:
out = bidirectional(out)
out

<tf.Tensor: shape=(1, 128), dtype=float32, numpy=
array([[-9.97816445e-04,  3.80099076e-03,  8.68839771e-03,
        -1.95693457e-03, -3.97455296e-05, -5.70358767e-04,
        -3.77196656e-03, -8.62555951e-03, -4.84917127e-03,
        -1.69709474e-02, -1.07297851e-02,  5.89317549e-03,
         2.60493136e-03,  4.11660830e-03,  7.34699017e-04,
         8.56950472e-04, -3.73841939e-03,  5.05867926e-03,
         1.36802280e-02, -5.21435589e-03,  1.38048781e-02,
         6.40078168e-03,  5.12945466e-03,  1.65308523e-03,
         5.47003094e-03,  4.84444667e-03,  7.43089151e-03,
         4.47930163e-03,  7.46450759e-03, -7.40424916e-03,
        -5.16483342e-05,  3.83321382e-03,  8.68863892e-04,
         6.44683978e-03,  6.80710515e-03, -6.51586009e-03,
         2.93811760e-03, -4.58461605e-03, -1.81733980e-03,
         1.53497811e-02, -3.19412816e-03, -2.20843678e-04,
         7.59002892e-03, -3.84386024e-03,  2.53942824e-04,
        -4.82280692e-03, -1.61946770e-02,  1.72614155e-03,
      

In [58]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(encoder.vocab_size, 64),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1)
])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, None, 64)          523840    
_________________________________________________________________
bidirectional_3 (Bidirection (None, None, 128)         66048     
_________________________________________________________________
bidirectional_4 (Bidirection (None, 64)                41216     
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 65        
Total params: 635,329
Trainable params: 635,329
Non-trainable params: 0
________________________________________________

In [59]:
X, t = next(iter(train_dataset.take(1)))
print(X)
print(t)

tf.Tensor(
[[  62    9    4   74 3416    6 3304  413 7968    8 1334    3 4608    2
    26   33 3245   24   10   16   10   17   19 1634    9 6805  330    2
  4038   34   44  425    1 6135 7961    7  786 4266  464  370    2   26
    15  620 4727    8    1 5907 7961    6  308   14  235   52    4  928
   371 6597   11 2982 8044    2  479   13    1  563   98    9 1527 7974
  2665 8039   24   10   16   10   17   19  188    9 7722 8045    3 3417
  7961 3003 1363  656    9 3429  234    2   20   59  168 5416 8044    2
    26    1 7814 7961 5050  363   48    1  108  170    9 7912 1338   43
     4   97  200 4018 7961 3647 4895  436   48   59  285  827 8044    3
   147  108 3165    2   87  159   33  382    7  754  302    6    1 4736
  7961    6   59 6660 7974  308 6163  874  731 8044    3  661    6    1
  1107  280    9 7894 7961    6  362 7974  443   95   67   97   11    1
   730   13    9 7032 7961   25 1910   26   61    6  465   11 3701 1478
  8044    3 7919 3734  124    2   12   80 1687 4294  

In [60]:
embedding = tf.keras.layers.Embedding(encoder.vocab_size, 64)

In [61]:
out = embedding(X)
out

<tf.Tensor: shape=(1, 283, 64), dtype=float32, numpy=
array([[[-0.04301181, -0.01161209,  0.00400028, ...,  0.04466926,
          0.04816457, -0.0170275 ],
        [-0.00789573, -0.03425347, -0.00430868, ...,  0.03422823,
         -0.01997321, -0.0161503 ],
        [-0.0062487 ,  0.04314747,  0.04811534, ..., -0.02057808,
         -0.02668605,  0.04205878],
        ...,
        [ 0.04444594, -0.01158798, -0.02603447, ..., -0.02020837,
          0.0344154 ,  0.01079761],
        [-0.02353125,  0.00099815,  0.01999016, ..., -0.00533328,
         -0.01106126,  0.01700855],
        [-0.03199371,  0.04913095, -0.01105053, ..., -0.03317032,
          0.03449178, -0.01146662]]], dtype=float32)>

In [63]:
bidirectional = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,  return_sequences=True))

In [64]:
out = bidirectional(out)
out

<tf.Tensor: shape=(1, 283, 128), dtype=float32, numpy=
array([[[ 7.5543616e-03,  2.3688262e-03,  2.5407907e-03, ...,
          5.6101712e-03,  6.3766297e-03,  2.4450503e-03],
        [ 6.6763991e-03,  6.9871251e-03, -5.1589399e-03, ...,
          8.5416147e-03,  2.8929231e-03,  1.5195498e-03],
        [ 3.4159217e-03,  1.2212612e-03, -3.2821676e-04, ...,
          4.2753520e-05, -3.2854762e-03,  5.0404770e-03],
        ...,
        [-5.7771844e-03, -1.8145925e-03,  5.6272745e-03, ...,
         -7.1272235e-03,  1.0151366e-03,  8.5844351e-03],
        [-8.1471559e-03, -2.1718862e-03, -2.1055001e-03, ...,
         -7.7146338e-03,  3.9796568e-03,  1.5647940e-02],
        [-8.3871782e-03, -4.8984010e-03, -2.2977621e-03, ...,
         -4.3367245e-03, -1.9046875e-04,  9.8884022e-03]]], dtype=float32)>

In [65]:
bidirectional2 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32))

In [66]:
out = bidirectional2(out)
out

<tf.Tensor: shape=(1, 64), dtype=float32, numpy=
array([[-3.1293831e-03, -1.9025865e-03,  4.0122122e-03, -7.8697354e-03,
        -2.6707093e-03, -3.7314554e-03, -5.3467401e-03,  6.7689624e-03,
         4.3838732e-03,  7.2056899e-04, -5.0442372e-03, -3.9336192e-03,
        -2.9937602e-03, -1.0267447e-03, -8.8486318e-03, -2.7576976e-03,
         1.3507295e-02, -6.8702386e-03, -3.0414150e-03, -2.4352972e-03,
        -1.6762698e-03, -1.5979625e-02,  6.9629797e-03,  6.6693206e-03,
         6.2476494e-03, -5.0151600e-03,  8.1077963e-03,  3.6814178e-03,
        -4.0519508e-03,  8.4482869e-03,  2.6353528e-03,  1.3888065e-03,
        -1.1800021e-03, -6.2227278e-04,  3.8472132e-04, -9.6335215e-03,
        -2.1441774e-03, -1.1113255e-03, -2.3558994e-05,  5.9463480e-03,
        -1.1599832e-03, -2.7276364e-03,  6.0444843e-04,  3.3455424e-03,
        -6.0578815e-05,  3.3877886e-04,  2.2616284e-03,  3.8239001e-03,
        -7.2138100e-03, -8.1754966e-05, -1.5340125e-05,  6.0172513e-04,
        -3.7130